# Solar Farm Applications in Nature Reserves(SSSI)

In [36]:
import leafmap
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, overlay
from shapely.geometry import Point
import matplotlib.pyplot as plt

from datetime import date
from dateutil.relativedelta import relativedelta




### Set the date range for the API call:


In [37]:

today = date.today()
today = today.strftime('%Y-%m-%d')
three_months_ago = date.today() + relativedelta(months=-3)
three_months_ago = three_months_ago.strftime('%Y-%m-%d')

### Get dataframe from API

In [38]:
# https://www.planit.org.uk/api/

df=pd.read_json('https://www.planit.org.uk/api/applics/json?start_date='+ three_months_ago +'&'+ 'end_date=' + today +'&search=solar&compress=on')


# extract new dataframe from dictionary of records

solar = pd.json_normalize(df['records'])
solar.head()

,name,uid,scraper_name,description,address,postcode,url,associated_id,app_size,app_state,...,other_fields.longitude,other_fields.decision_date,other_fields.appeal_status,other_fields.applicant_company,other_fields.decision_issued_date,other_fields.decision_published_date,other_fields.permission_expires_date,other_fields.appeal_result,other_fields.meeting_date,other_fields.n_dwellings
0,Crawley/CR/2021/0573/192,CR/2021/0573/192,Crawley,CERTFICATE OF LAWFULNESS FOR INSTALLATION AND IMPLEMENTATION OF ROOF MOUNTED 273KWP SOLAR PV ARRAY AT VODAFONE CRAWLEY SITE,"SITE B, VECTOR 2, NEWTON ROAD, NORTHGATE, CRAWLEY",None,https://planningregister.crawley.gov.uk/Planning/Display/CR/2021/0573/192,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wiltshire/PL/2021/08598,PL/2021/08598,Wiltshire,"Erection of agricultural barn for use of storage to house; farming equipment ie tractor/mower, log drying area, wash room. To include solar panelling on the roof for powering of equipment.","Land west of Woodlands Way, Woodlands Road, Mere, BA12 6JT",BA12 6JT,https://development.wiltshire.gov.uk/pr/s/planning-application/a0i3z000016HXh2AAG,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Reigate/21/02530/LBC,21/02530/LBC,Reigate,Installation of six solar panels on south-facing roof of kitchen/diner and master bedroom.,Camellia Cottage 154 Dovers Green Road Reigate Surrey RH2 8BZ,RH2 8BZ,https://planning.reigate-banstead.gov.uk/online-applications/applicationDetails.do?activeTab=summary&keyVal=QZW3GQMVKUR00,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Reigate/21/02529/HHOLD,21/02529/HHOLD,Reigate,Installation of six solar panels on south-facing roof of kitchen/diner and master bedroom.,Camellia Cottage 154 Dovers Green Road Reigate Surrey RH2 8BZ,RH2 8BZ,https://planning.reigate-banstead.gov.uk/online-applications/applicationDetails.do?activeTab=summary&keyVal=QZW3GPMVKUQ00,None,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wycombe/21/07948/PNP14J,21/07948/PNP14J,Wycombe,"Prior notification application (Part 14, Class J) for installation of 93.48kWp solar panels on roof",Unit C 2 Thomas Road Wooburn Industrial Estate Wooburn Green Buckinghamshire HP10 0PE,HP10 0PE,https://publicaccess.wycombe.gov.uk/idoxpa-web/applicationDetails.do?keyVal=R08QXVSC0O500&activeTab=summary,93.48kWp,Small,Undecided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
solar.shape
#solar.to_csv('solar.csv')

(1034, 83)

### Filtering the data

In [40]:
solar['description'] = solar['description'].str.lower() 

# exclude (!consider risks)

# solar = solar[(~solar['description'].str.contains('roof'))] #residential
# solar.shape

In [42]:
# include

mylist = ['watts', 'mega', 'solar park', 'land '] # land is poor discriminator!
pattern = '|'.join(mylist)

solar_parks = solar.loc[solar['description'].str.contains(pattern)]

In [43]:
solar_parks.shape

(50, 83)

In [44]:
pd.set_option('max_colwidth', 400)

solar_parks.description.head(10)

16                                                                                                                                                                                                                                                                                                                retention of change of use of land for siting of 5.5kw ground mounted solar panel photovoltaic array
19                                                                                                                  environmental impact assessment (eia) screening opinion in accordance with regulation 6 of the town and country planning (environmental impact assessment) regulations 2017, as amended1 from hart district council (hdc) with regard to the proposed solar farm on land at kiln fields, heckfield
62                                                                                                                                                            discharge condition 9 (lands

### Issues with missing spatial columns

In [45]:
# get me the entries without any spatial data
missing_spatials = ['postcode', 'location_x', 'location_y', 'other_fields.lat', 'other_fields.lng', 'other_fields.easting', 'other_fields.northing']


solar_invisible = solar[solar[missing_spatials].isna().all(1)]
print(solar_invisible.shape)
#solar_invisible.shape

(94, 83)


In [46]:
# make a csv of cases with hyperlinks for their geography

#header = ['postcode', 'description', 'location_x', 'location_y', 'other_fields.lat', 'other_fields.lng', 'other_fields.easting', 'other_fields.northing']
solar_invisible.to_csv('solar_invisible.csv') #columns = header

### Fix geometries for cases that can be fixed quickly

In [47]:
# choosing column with max chance of simple fix for geography
solar['location.coordinates'].iloc[68]

[-2.741489, 50.81544]

In [48]:
# split out location coordinates into two cols

# handle missing values (i.e. possible cases will be missing)
dropped = solar['location.coordinates'].dropna()
solar[['X', 'Y']] = pd.DataFrame(dropped.tolist(), index=dropped.index)

In [49]:
solar.shape

(1034, 85)

In [50]:
# drop cases where geography is missing

solar = solar.dropna(subset = ['X', 'Y'])

In [51]:
solar.shape

(923, 85)

### Get the Nature reserve mapping SSSIs

In [52]:
# attribution: UK Department of Environment Food & Rural Affairs
# https://environment.data.gov.uk/DefraDataDownload/?mapService=NE/SitesOfSpecialScientificInterestEngland&Mode=spatial

SSSIs = gpd.read_file("data/Sites_of_Special_Scientific_Interest_England.shp")
#SSSIs.set_index(['geometry'])

In [53]:
SSSIs.head(1)


,sssi_name,sssi_area,easting,northing,latitude,longitude,reference,status,gid,ensisid,gis_file,area,easting0,northing0,gis_date,version,st_area_sh,st_perimet,geometry
0,Allen Confluence Gravels,4.777198,379993.104106,558767.08019,54:55:23N,2:19:10W,NY799587,Notified,1003435.0,1005624.0,None,4.777198,379993.104106,558767.08019,20031218,1.0,47771.979838,1957.148648,"POLYGON ((380021.211 558598.082, 380011.202 558601.101, 380001.868 558605.190, 379985.798 558610.799, 379973.497 558613.098, 379956.199 558614.198, 379952.299 558614.098, 379936.699 558612.298, 379914.899 558609.199, 379896.702 558609.199, 379888.202 558609.699, 379878.901 558610.699, 379829.398 558618.876, 379813.304 558614.897, 379784.899 558610.479, 379769.596 558608.299, 379763.899 558608...."


In [54]:
# SSSIs.dtypes

In [55]:
# SSSIs.crs

In [56]:
# SSSIs.total_bounds

### Working towards plot of all applications and likely solar park applications in SSSIs

In [57]:


print('Solar applications: complete list =', solar.shape[0], ', filtered list =', solar_parks.shape[0])

Solar applications: complete list = 923 , filtered list = 50


In [58]:
solarGDF = gpd.GeoDataFrame(solar, geometry=gpd.points_from_xy(solar['X'], solar['Y']))  # crs={'init':'epsg:27700'}
solarGDF.shape

(923, 86)

In [59]:
# solarGDF.total_bounds

In [60]:
solarGDF = solarGDF.set_crs(epsg=4326, inplace=True)


In [61]:
# solarGDF.total_bounds

In [62]:
solar27700 = solarGDF.to_crs(epsg=27700)
solar27700.total_bounds

array([101522.99126112, -76022.26359639, 646593.41325876, 959466.57531758])

In [63]:
# solarGDF['geometry'] = solarGDF.geometry.buffer(50000)

In [64]:
# solarGDF.head(1)

In [65]:
# solar27700.shape

In [66]:
# fig, ax = plt.subplots(figsize=(20, 16))

# SSSIs.plot(ax = ax, edgecolor='black',  )

# solar27700.plot(ax=ax, marker='o', color='red', markersize=2)

# plt.show()

In [67]:
pointinpolys = gpd.sjoin(SSSIs, solar27700, op='contains', how='inner' ) # 

In [68]:
pointinpolys.shape

(2, 105)

In [69]:
polyswithpoints = gpd.sjoin(solar27700, SSSIs, op='within', how='inner')

In [34]:
polyswithpoints.shape

(2, 105)

In [35]:
m = leafmap.Map(center=[50.5,-4], zoom=8, height="700px", width="700px")
m.add_gdf(pointinpolys, layer_name="SSSIs with solar planning applications")
m.add_gdf(polyswithpoints, layer_name="solar planning applications")
m

# this map currently plots any application for 'solar' that lands inside an SSSI

Map(center=[50.5, -4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…


### Try to buffer the points for solar applications just ouside nature reserves - NOT WORKING YET:

In [ ]:
bufferpoint = solar27700

bufferpoint['geometry'] = solar27700.buffer(200)

In [ ]:
# buffer1km['geometry']

In [ ]:


polybuffered = overlay(SSSIs, bufferpoint, how='intersection')

In [ ]:
polybuffered.shape

In [ ]:
pointbuffered = gpd.sjoin(SSSIs, buffer1km, op='contains', how='inner' )

In [ ]:
pointbuffered.shape

In [ ]:
m = leafmap.Map(center=[54,-2], zoom=6, height="800px", width="450px")
m.add_gdf(SSSIs, layer_name="SSSIs")
m

In [ ]:
cols = solar.columns

In [ ]:
SSSI_example = SSSIs.loc[SSSIs['sssi_name'] == 'Taw-Torridge Estuary']
# SSSI_example.crs
# SSSI_example.head()

In [ ]:
fake_solar = pd.read_csv('fake_applications.csv')

In [ ]:
fake_solar.head()

In [ ]:
fakeGDF = gpd.GeoDataFrame(fake_solar, geometry=gpd.points_from_xy(fake_solar['X'], fake_solar['Y']))  # crs={'init':'epsg:27700'}
fakeGDF.shape

In [ ]:
fakeGDF = fakeGDF.set_crs(epsg=4326, inplace=True)


In [ ]:
fakeGDF.crs == SSSI_example.crs

In [ ]:
fakeGDF.total_bounds

In [ ]:
SSSI_example.total_bounds

In [ ]:
fake27700 = fakeGDF.to_crs(epsg=27700)

In [ ]:
SSSI27700 = SSSI_example.to_crs(epsg=27700)

In [ ]:
SSSI27700.geometry

In [ ]:
fake27700.geometry

In [ ]:
fake27700.total_bounds

In [ ]:
SSSI27700.total_bounds

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))

#polyswithpoints.plot(ax = ax, edgecolor='black',  )

fake27700.plot(ax=ax, marker='o', color='red', markersize=2)
SSSI27700.plot(ax=ax)

plt.show()